In [1]:
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession.builder \
    .appName("MercadoPago") \
    .getOrCreate()

In [5]:
#historial de 1 mes de pagos realizados por los usuarios
pays_path = '/home/jovyan/work/spark_env/pays.csv'

#historial de 1 mes de value props que fueron mostradas a cada usuario
prints_path = '/home/jovyan/work/spark_env/prints.json'

#historial de 1 mes de value props que fueron clickeadas por un usuario
taps_path = '/home/jovyan/work/spark_env/taps.json'

In [6]:
paysDF = spark.read.csv(path=pays_path, header=True)

In [12]:
from pyspark.sql.functions import to_date, when, count, window, desc, asc, sum
from pyspark.sql.types import IntegerType, StringType

In [13]:
paysDF = paysDF.withColumn('pay_date', to_date(paysDF['pay_date'], 'yyyy-MM-dd'))
paysDF = paysDF.withColumn('total', paysDF['total'].cast(IntegerType()))
paysDF = paysDF.withColumn('user_id', paysDF['user_id'].cast(IntegerType()))
paysDF = paysDF.withColumn('value_prop', paysDF['value_prop'].cast(StringType()))

In [14]:
paysDF.printSchema()

root
 |-- pay_date: date (nullable = true)
 |-- total: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- value_prop: string (nullable = true)



In [15]:
paysDF.show()

+----------+-----+-------+------------------+
|  pay_date|total|user_id|        value_prop|
+----------+-----+-------+------------------+
|2020-11-01|    7|  35994|        link_cobro|
|2020-11-01|   37|  79066|cellphone_recharge|
|2020-11-01|   15|  19321|cellphone_recharge|
|2020-11-01|   26|  19321|        send_money|
|2020-11-01|   35|  38438|        send_money|
|2020-11-01|   20|  85939|         transport|
|2020-11-01|   74|  14372|           prepaid|
|2020-11-01|   31|  14372|        link_cobro|
|2020-11-01|   83|  65274|         transport|
|2020-11-01|   93|  65274|           prepaid|
|2020-11-01|   37|  97428|        link_cobro|
|2020-11-01|   26|  82163|        link_cobro|
|2020-11-01|   92|   9816|        send_money|
|2020-11-01|  122|   9816|           prepaid|
|2020-11-01|   83|  28929|           prepaid|
|2020-11-01|  136|  97275|        link_cobro|
|2020-11-01|   17|  85001|cellphone_recharge|
|2020-11-01|   41|  85001|        link_cobro|
|2020-11-01|   31|     42|        